In [ ]:
# -*- coding: utf-8 -*-
"""
PRONÓSTICO DIARIO A 1 AÑO POR SUBÁREA (SIN TEMPERATURA)
=======================================================

Objetivo
--------
Pronosticar la demanda eléctrica **diaria** (no horaria) por subárea, a **365 días**.
Se usan 5 años diarios simulados, y un enfoque robusto para largo plazo:

- Modelo base: **SARIMAX** con estacionalidad semanal (periodo 7) vía (0,1,1)(0,1,1,7),
  y **estacionalidad anual** capturada con **términos de Fourier** (periodo ~365.25).
- Exógenas de calendario: **día-de-semana (dummies)** y **festivos**.
- Evaluación: **hold-out del último año**: se entrena con años previos y se pronostica
  365 días para comparar vs observaciones reales del último año.
- Baseline: **naïve anual** (valor de hace 365 días).
- Pronóstico final: 365 días futuros (después del último punto observado) con las exógenas futuras.

Base teórica (resumen)
----------------------
1) **SARIMA/SARIMAX**:
   - Parte ARIMA: diferenciar para estacionariedad en media/tendencia.
   - Parte SARIMA: diferencia estacional de periodo 7 para patrón semanal.
   - **Orden elegido**: (0,1,1)(0,1,1,7) es un "caballo de batalla" habitual que
     funciona bien para consumo diario con semana laboral vs fin de semana.

2) **Fourier anual como exógena**:
   - Para estacionalidad larga (≈ 365 días), usar términos sin/cos
     sin crear 365 dummies: con K armónicos capturamos variaciones suaves anuales.
     ŷ_t ≈ β0 + Σ_{k=1..K} [a_k sin(2π k t / 365.25) + b_k cos(2π k t / 365.25)] + ...
   - Aquí usamos K=3 (puedes subir a 5 si la forma anual es compleja).

3) **Exógenas de calendario**:
   - Dummies de día-de-semana y **festivo** (0/1) ayudan a la forma semanal y eventos fijos.

4) **Validación para largo plazo**:
   - Evitar recursión hora-a-hora. Aquí el horizonte es diario y el modelo pronostica
     directamente 365 pasos usando su estructura probabilística + exógenas futuras.

Cómo adaptar a datos reales
---------------------------
- Reemplaza la simulación por tu DataFrame con columnas:
  `date` (diaria), `subarea` (string), `demand` (float), `holiday` (0/1).
- Si tu histórico es horario, **agrega**: demanda diaria = suma/avg por día y subárea.
- Mantén el pipeline de Fourier (anual), dummies (dow) y festivos.

"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import pi
from itertools import product

from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error


# =========================
# 0) PARÁMETROS GENERALES
# =========================
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

N_SUBAREAS  = 6
START_DATE  = "2020-01-01"
END_DATE    = "2024-12-31"    # ~5 años diarios
TEST_DAYS   = 365             # último año para evaluación
FORECAST_H  = 365             # horizonte futuro (1 año)

FOURIER_PERIOD = 365.25
K_FOURIER      = 3            # armónicos anuales (sube a 5 si necesitas más detalle)


# =========================
# 1) SIMULACIÓN DIARIA
# =========================
def simulate_daily_demand(idx, subarea_id):
    """
    Simula demanda DIARIA sin temperatura:
    - Nivel por subárea
    - Efecto semanal (dow): menor en sáb-dom
    - Estacionalidad anual suave (Fourier) + leve tendencia
    - Festivos (reducción)
    - Ruido
    """
    dow = idx.dayofweek.values    # 0..6
    doy = idx.dayofyear.values

    # Nivel base distinto por subárea
    base = 220 + 18*(subarea_id - 3)

    # Semana: bajón en fin de semana
    weekly = -18 * (dow >= 5).astype(float)

    # Estacionalidad anual suave con 2 armónicos "duros" + ruido
    annual = 10*np.sin(2*pi*doy/365.25) + 6*np.cos(2*pi*2*doy/365.25)

    # Tendencia ligera
    t = np.arange(len(idx))
    trend = 0.003 * t

    # Festivos (mismo set fijo a modo de ejemplo)
    holidays = pd.Series(0, index=idx)
    for y in np.unique(idx.year):
        for m, d in [(1,1), (5,1), (12,25)]:
            try:
                holidays[pd.Timestamp(y, m, d)] = 1
            except:
                pass
    holidays = holidays.reindex(idx, fill_value=0).astype(int).values
    holiday_effect = -25 * holidays

    # Ruido
    noise = np.random.normal(0, 5, size=len(idx))

    y = base + weekly + annual + trend + holiday_effect + noise
    return np.maximum(y, 10), holidays


def build_daily_panel():
    """
    DataFrame largo:
      date | subarea | demand | holiday
    """
    idx = pd.date_range(START_DATE, END_DATE, freq="D")
    dfs = []
    for s in range(1, N_SUBAREAS+1):
        demand, holidays = simulate_daily_demand(idx, s)
        df = pd.DataFrame({
            "date": idx,
            "subarea": f"S{s}",
            "demand": demand,
            "holiday": holidays
        })
        dfs.append(df)
    return pd.concat(dfs, ignore_index=True)

data = build_daily_panel().sort_values(["subarea", "date"]).reset_index(drop=True)


# =========================
# 2) EXÓGENAS (FOURIER + CALENDARIO)
# =========================
def fourier_terms(dates, period=FOURIER_PERIOD, K=K_FOURIER, prefix="year"):
    """
    Términos de Fourier (sin/cos) para estacionalidad periódica (aquí anual).
    No explotan dimensionalidad como 365 dummies y suelen generalizar mejor.
    """
    t = np.arange(len(dates))
    out = {}
    for k in range(1, K+1):
        out[f"{prefix}_sin_{k}"] = np.sin(2*pi*k*t/period)
        out[f"{prefix}_cos_{k}"] = np.cos(2*pi*k*t/period)
    return pd.DataFrame(out, index=dates)

def build_exog(df):
    """
    Exógenas de calendario + Fourier:
    - Dummies de día-de-semana (dow_1..dow_6, con base dow_0)
    - Festivo (0/1)
    - Fourier anual (K armónicos)
    """
    exog = pd.DataFrame(index=df.index)
    # Día de semana (0..6). drop_first=True para evitar colinealidad.
    dow = pd.get_dummies(df["date"].dt.dayofweek, prefix="dow", drop_first=True).astype(int)
    exog = pd.concat([exog, dow], axis=1)

    # Festivo (ya viene en df)
    exog["holiday"] = df["holiday"].astype(int).values

    # Fourier anual
    ft = fourier_terms(df["date"], period=FOURIER_PERIOD, K=K_FOURIER, prefix="yr")
    ft.index = df.index
    exog = pd.concat([exog, ft], axis=1)

    return exog


# =========================
# 3) SPLIT: TRAIN / TEST
# =========================
last_day   = data["date"].max()
test_start = last_day - pd.Timedelta(days=TEST_DAYS) + pd.Timedelta(days=1)  # inicio del año hold-out


# =========================
# 4) ENTRENAMIENTO Y TEST
# =========================
def fit_sarimax_daily(y, exog, order=(0,1,1), seasonal_order=(0,1,1,7)):
    """
    Ajusta un SARIMAX(y ~ exog) con:
    - Diferencia no estacional (d=1) para tendencia
    - Diferencia estacional semanal (D=1, s=7) para componente semanal
    - Componentes MA (no estacional y estacional) para captar autocorrelaciones residuales
    Esta especificación es robusta y suele rendir bien con consumo/energía diarios.
    """
    model = SARIMAX(
        endog=y,
        exog=exog,
        order=order,
        seasonal_order=seasonal_order,
        enforce_stationarity=False,
        enforce_invertibility=False
    )
    res = model.fit(disp=False)
    return res


def prep_exog_consistent(train_df, test_df, future_df):
    """
    Asegura que TRAIN/TEST/FUTURE tengan **exactamente** las mismas columnas exógenas
    (p. ej., si en un segmento falta una categoría de dow).
    """
    ex_tr = build_exog(train_df).copy()
    ex_te = build_exog(test_df).copy()
    ex_fu = build_exog(future_df).copy()

    cols = sorted(set(ex_tr.columns) | set(ex_te.columns) | set(ex_fu.columns))
    ex_tr = ex_tr.reindex(columns=cols, fill_value=0)
    ex_te = ex_te.reindex(columns=cols, fill_value=0)
    ex_fu = ex_fu.reindex(columns=cols, fill_value=0)
    return ex_tr, ex_te, ex_fu


# =========================
# 5) LOOP POR SUBÁREA
# =========================
results = []
forecasts = []

for sub in data["subarea"].unique():
    df_sub = data[data["subarea"] == sub].copy().reset_index(drop=True)

    # Particiones
    df_train = df_sub[df_sub["date"] < test_start].copy()
    df_test  = df_sub[df_sub["date"] >= test_start].copy()  # último año (para evaluar)

    # Exógenas consistentes
    ex_tr, ex_te, ex_fu_dummy = prep_exog_consistent(df_train, df_test, df_test)  # futuro real lo armamos luego

    # Ajustar SARIMAX en TRAIN
    y_tr = df_train["demand"].values
    model_res = fit_sarimax_daily(y_tr, ex_tr)

    # PRONÓSTICO DEL AÑO HOLD-OUT (TEST) para evaluar
    yhat_test = model_res.predict(start=len(y_tr), end=len(y_tr)+len(df_test)-1, exog=ex_te)
    y_true    = df_test["demand"].values

    mape = mean_absolute_percentage_error(y_true, yhat_test) * 100
    rmse = np.sqrt(mean_squared_error(y_true, yhat_test))

    # Baseline naïve anual (mismo día del año pasado)
    # Tomamos el valor de hace 365 días en TRAIN si existe; alineamos tamaños
    if len(df_train) >= 365:
        naive_last_year = df_train["demand"].values[-365:][:len(df_test)]
        mape_naive = mean_absolute_percentage_error(y_true, naive_last_year) * 100
    else:
        mape_naive = np.nan

    results.append({"subarea": sub, "MAPE_%": mape, "RMSE": rmse, "MAPE_naive_%": mape_naive})

    # =========================
    # 6) PRONÓSTICO FUTURO 365 DÍAS
    # =========================
    last_obs = df_sub["date"].max()
    future_idx = pd.date_range(last_obs + pd.Timedelta(days=1), periods=FORECAST_H, freq="D")
    df_future = pd.DataFrame({
        "date": future_idx,
        "subarea": sub,
        # festivos futuros simples (reemplaza por tu calendario real)
        "holiday": 0
    })
    for y in np.unique(df_future["date"].dt.year):
        for m, d in [(1,1), (5,1), (12,25)]:
            ts = pd.Timestamp(y, m, d)
            if ts in set(df_future["date"]):
                df_future.loc[df_future["date"] == ts, "holiday"] = 1

    # Preparar exógenas para FUTURE (mismas columnas)
    ex_tr2, ex_te2, ex_fu = prep_exog_consistent(df_train, df_test, df_future)

    # Reentrenar con TODO el histórico antes del futuro (opcional pero recomendable)
    ex_full = build_exog(df_sub).reindex(columns=ex_tr2.columns, fill_value=0)
    model_res_full = fit_sarimax_daily(df_sub["demand"].values, ex_full)

    # Pronóstico 365 días FUTURO
    yhat_future = model_res_full.predict(start=len(df_sub), end=len(df_sub)+FORECAST_H-1, exog=ex_fu)

    forecasts.append(pd.DataFrame({
        "date": future_idx,
        "subarea": sub,
        "forecast": yhat_future
    }))

# Métricas por subárea + global
metrics_df = pd.DataFrame(results)
metrics_df.loc[len(metrics_df)] = {
    "subarea": "GLOBAL",
    "MAPE_%": metrics_df["MAPE_%"].mean(),
    "RMSE": metrics_df["RMSE"].mean(),
    "MAPE_naive_%": metrics_df["MAPE_naive_%"].mean()
}

print("\nMÉTRICAS HOLD-OUT (último año):")
print(metrics_df.round(3))

forecast_df = pd.concat(forecasts, ignore_index=True).sort_values(["subarea","date"])
print("\nHEAD del pronóstico futuro (primeras 10 filas):")
print(forecast_df.head(10))

# (Opcional) Guardar a CSV
# forecast_df.to_csv("forecast_diario_1anio_por_subarea.csv", index=False)

# (Opcional) Gráfica rápida para una subárea
# sub_demo = "S1"
# df_sub = data[data["subarea"] == sub_demo].copy().reset_index(drop=True)
# df_train = df_sub[df_sub["date"] < test_start]
# df_test  = df_sub[df_sub["date"] >= test_start]
# ex_tr, ex_te, _ = prep_exog_consistent(df_train, df_test, df_test)
# res = fit_sarimax_daily(df_train["demand"].values, ex_tr)
# yhat_test = res.predict(start=len(df_train), end=len(df_train)+len(df_test)-1, exog=ex_te)
# plt.figure(figsize=(12,4))
# plt.plot(df_test["date"], df_test["demand"], label="Real")
# plt.plot(df_test["date"], yhat_test, label="Pronóstico")
# plt.title(f"Subárea {sub_demo} – Último año (hold-out)")
# plt.xlabel("Fecha"); plt.ylabel("Demanda diaria"); plt.legend(); plt.tight_layout(); plt.show()

